# Advanced prompting
- using a personal API key
- in run thread : is needed (assistant_id, thread_id, prompt, esg_report_file_path, regulations_file_path)
- ESG report and regulation are attached once (!= None:)



In [3]:
from openai import OpenAI
import pandas as pd

client = OpenAI(
  api_key="personal_API_key"
)

def runThread(assistant_id, thread_id, prompt, esg_report_file_path, regulations_file_path):
  assistant = client.beta.assistants.retrieve(
    assistant_id=assistant_id
  )
  thread = client.beta.threads.retrieve(
    thread_id=thread_id
  )

  # Upload the user provided file to OpenAI
  attachments = []

  if esg_report_file_path != None:
    esg_report_file = client.files.create(
      file=open(esg_report_file_path, "rb"), purpose="assistants"
    )
    attachments.append({ "file_id": esg_report_file.id, "tools": [{"type": "file_search"}] })
  
  if regulations_file_path != None:
    regulations_file = client.files.create(
      file=open(regulations_file_path, "rb"), purpose="assistants"
    )
    attachments.append({ "file_id": regulations_file.id, "tools": [{"type": "file_search"}] })


  # add message to thread
  client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
    attachments=attachments
  )

  run = client.beta.threads.runs.create_and_poll(
      thread_id=thread.id,
      assistant_id=assistant.id,
  )

  threadMessages = client.beta.threads.messages.list(thread.id)

  assistantMessage = next((i for i in threadMessages if i.role == "assistant" and i.run_id == run.id), None)
  return assistantMessage.content[0].text.value

# Definiton of  input & output 
Difference to zero-shot:
-using the created thread. id for the input prompts (prompt_1,prompt_2,prompt_3) & outputs ('answer_1,answer_2,answer_3)
-creating one thread per row from the input file (analysis/fewshot.xlsx)








In [4]:
import pandas as pd
#Origin input excel with prompt and assistant ID
excel_file_path = "analysis/fewshot.xlsx"

# Load the Excel file
df = pd.read_excel(excel_file_path, sheet_name='Sheet1', keep_default_na=False)

# Iterate over DataFrame rows one after another so that all prompts of a company are answered in one thread
for i in df.index:
    if df.loc[i, 'answer_1'] == '':
        thread = client.beta.threads.create()

        answer1 = runThread(
            assistant_id=df.loc[i, 'assistant_id'],
            thread_id=thread.id,
            prompt=df.loc[i, 'prompt_1'],
            esg_report_file_path='./esg_reportings/' + df.loc[i, 'file_name'],
            regulations_file_path = 'G1-3-regulation.pdf' 
        )
        df.loc[i, 'answer_1'] = answer1

        if df.loc[i, 'answer_2'] == '':
            answer2 = runThread(
                assistant_id=df.loc[i, 'assistant_id'],
                thread_id=thread.id,
                prompt=df.loc[i, 'prompt_2'],
                esg_report_file_path= None,
                regulations_file_path = None
            )
            df.loc[i, 'answer_2'] = answer2

            if df.loc[i, 'answer_3'] == '':
                answer3 = runThread(
                    assistant_id=df.loc[i, 'assistant_id'],
                    thread_id=thread.id,
                    prompt=df.loc[i, 'prompt_3'],
                    esg_report_file_path= None,
                    regulations_file_path = None
                )
                df.loc[i, 'answer_3'] = answer3

        # Save the DataFrame back to the Excel file used
        df.to_excel(excel_file_path, index=False, sheet_name='Sheet1')

#show dataframe
df


,file_name,assistant_id,prompt_1,answer_1,prompt_2,answer_2,prompt_3,answer_3
0,Adidas.pdf,asst_HtUbDv6XHKlxaT9ElaKwzpP1,Provide a comprehensive analysis as short and ...,### ESG Report Analysis: adidas AG\n\n#### Pre...,Provide a short and precise comprehensive anal...,### ESG Report Analysis: adidas AG\n\n#### Inv...,Provide a comprehensive analysis of the upload...,### ESG Report Analysis: adidas AG\n\n#### Tra...
1,Airbus.pdf,asst_HtUbDv6XHKlxaT9ElaKwzpP1,Provide a comprehensive analysis as short and ...,**Company Name:** Airbus\n\n**Prevention and I...,Provide a short and precise comprehensive anal...,**Company Name:** Airbus\n\n**Investigation In...,Provide a comprehensive analysis of the upload...,**Company Name:** Airbus\n\n**Training Program...
2,Allianz.pdf,asst_HtUbDv6XHKlxaT9ElaKwzpP1,Provide a comprehensive analysis as short and ...,The analysis of the Allianz Group Sustainabili...,Provide a short and precise comprehensive anal...,The Allianz Group Sustainability Report 2022 o...,Provide a comprehensive analysis of the upload...,The Allianz Group Sustainability Report 2022 o...
3,BASF.pdf,asst_HtUbDv6XHKlxaT9ElaKwzpP1,Provide a comprehensive analysis as short and ...,### Analysis of BASF ESG Report on Corruption ...,Provide a short and precise comprehensive anal...,### Analysis of BASF ESG Report on Corruption ...,Provide a comprehensive analysis of the upload...,### Analysis of BASF ESG Report on Corruption ...
4,Bayer.pdf,asst_HtUbDv6XHKlxaT9ElaKwzpP1,Provide a comprehensive analysis as short and ...,**Analysis of Bayer's ESG Report on Corruption...,Provide a short and precise comprehensive anal...,**Analysis of Bayer's ESG Report on Investigat...,Provide a comprehensive analysis of the upload...,**Analysis of Bayer's ESG Report on Corruption...
5,Beiersdorf.pdf,asst_HtUbDv6XHKlxaT9ElaKwzpP1,Provide a comprehensive analysis as short and ...,### Analysis of Beiersdorf's ESG Report on Cor...,Provide a short and precise comprehensive anal...,### Comprehensive Analysis of Beiersdorf's ESG...,Provide a comprehensive analysis of the upload...,### Comprehensive Analysis of Beiersdorf's ESG...
6,BMW.pdf,asst_HtUbDv6XHKlxaT9ElaKwzpP1,Provide a comprehensive analysis as short and ...,The ESG report analyzed is from BMW Group and ...,Provide a short and precise comprehensive anal...,"The ESG report analyzed is from BMW Group, and...",Provide a comprehensive analysis of the upload...,The ESG report analyzed is from BMW Group. In ...
7,Brenntag.pdf,asst_HtUbDv6XHKlxaT9ElaKwzpP1,Provide a comprehensive analysis as short and ...,**ESG Report Analysis - Brenntag SE**\n\n**Com...,Provide a short and precise comprehensive anal...,**ESG Report Analysis - Brenntag SE**\n\n**Inv...,Provide a comprehensive analysis of the upload...,**ESG Report Analysis - Brenntag SE**\n\n**Tra...
8,Continental.pdf,asst_HtUbDv6XHKlxaT9ElaKwzpP1,Provide a comprehensive analysis as short and ...,**Analysis of Continental AG's ESG Report:**\n...,Provide a short and precise comprehensive anal...,**Analysis of Continental AG's ESG Report:**\n...,Provide a comprehensive analysis of the upload...,**Analysis of Continental AG's ESG Report:**\n...
9,Covestro.pdf,asst_HtUbDv6XHKlxaT9ElaKwzpP1,Provide a comprehensive analysis as short and ...,The ESG report analyzed is from Covestro. \n\n...,Provide a short and precise comprehensive anal...,**Comprehensive Analysis and Compliance Check ...,Provide a comprehensive analysis of the upload...,The analysis of Covestro's ESG report concerni...
